###Diabetic Retinopathy Detection
In this project, classification of retinal images for the presence of diabetic retinopathy is done.The dataset is taken from the kaggle competition (https://www.kaggle.com/c/diabetic-retinopathy-detection).Run the following cell to load files 

In [1]:
from os.path import isfile,isdir
import glob
import os
from PIL import ImageEnhance
from PIL import Image
import pandas as pd
import numpy as np

# ='/Users/hmakam200/Desktop/ramya_data/sample_images/onlyImage/onlyImage.csv'
#import csv
#images=[]
#for file in glob.glob("F:/Diabetic Retinopathy/TrainDataPreProcessed/FinalData/*.jpeg"):
 #   im=Image.open(file)
    #arr=list(im.getdata())
    #images.append(arr)
    #im.close()
#sort the labels file using image as value
#labels=pd.read_csv("F:/Diabetic Retinopathy/labels.csv")


In [15]:
import pickle

data=pickle.load(open("F:/Udacity ML/git/machine-learning/projects/image-classification/preprocess_batch_5.p","rb"))
print (data[0].shape)

(9000, 32, 32, 3)


In [21]:

#http://ischlag.github.io/2016/11/07/tensorflow-input-pipeline-for-large-datasets/
#Loading data
import tensorflow as tf
import numpy as np
import threading

r=np.arange(0.0,100003.0)
print("r array - one dimensional")
print (r[0])
print (r[10])
raw_data=np.dstack((r,r,r,r))[0]
print ("Raw data - 2 dimensional")
print (raw_data[0])
print (raw_data[90])
print (raw_data.shape)
print ("raw target - 2 dimensional")
raw_target=np.array([[1,0,0]]*100003)
print (raw_target[0])
print (raw_target[90])
print(raw_target.shape)

r array - one dimensional
0.0
10.0
Raw data - 2 dimensional
[ 0.  0.  0.  0.]
[ 90.  90.  90.  90.]
(100003, 4)
raw target - 2 dimensional
[1 0 0]
[1 0 0]
(100003, 3)


In [24]:
#Build a FIFO Queue


queue_input_data=tf.placeholder(tf.float32,shape=[20,4])
queue_input_target=tf.placeholder(tf.float32,shape=[20,3])


queue=tf.FIFOQueue(capacity=50,dtypes=[tf.float32,tf.float32],shapes=[[4],[3]])

enqueue_op=queue.enqueue_many([queue_input_data,queue_input_target])
dequeue_op=queue.dequeue()

#Now we can already continue building our input pipeline.
#After performing some preprocessing on the dequeued data 
#we can group them into a batch and use a session in order to draw the next batch of samples from our input pipeline.
#But before we can do that,
#we have to start a thread that will fill our queue object by calling queue.enqueue_many with data from our numpy data. 
#Here, instead of reading from our simple numpy data array
#you could also access a database, a network source, or a big file which you cannot load fully into memory.
#Notice that I loop endlessly in order to keep up a stream of incoming data.
#Don’t worry about shuffeling here you can use tf.train.shuffle_batch instead of tf.train.batch.

In [32]:
#tensorflow recommendation 
#capacity=min_after_dequeue+(num_threads + a small safety margin)*batch_size
data_batch,target_batch=tf.train.shuffle_batch(dequeue_op,batch_size=15,capacity=40)

In [33]:
#start the threads
#Now the only thing that is missing are the queue runner threads for our tf.train.batch.
def enqueue(sess):
    under = 0
    max = len(raw_data)
    while True:
        print("starting to write into queue")
        upper = under + 20
        print("try to enqueue ", under, " to ", upper)
        if upper <= max:
            curr_data = raw_data[under:upper]
            curr_target = raw_target[under:upper]
            under = upper
        else:
            rest = upper - max
            curr_data = np.concatenate((raw_data[under:max], raw_data[0:rest]))
            curr_target = np.concatenate((raw_target[under:max], raw_target[0:rest]))
            under = rest
            
        sess.run(enqueue_op, feed_dict={queue_input_data: curr_data,queue_input_target: curr_target})
        print("added to the queue")
    print("finished enqueueing")

sess = tf.Session()
enqueue_thread = threading.Thread(target=enqueue, args=[sess])
enqueue_thread.isDaemon()
enqueue_thread.start()

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord, sess=sess)



starting to write into queue
try to enqueue  0  to  20
added to the queue
starting to write into queue
try to enqueue  20  to  40
added to the queue
starting to write into queue
try to enqueue  40  to  60
added to the queue
starting to write into queue
try to enqueue  60  to  80
added to the queue
starting to write into queue
try to enqueue  80  to  100


In [34]:
#Now, the next batch for our model should be ready and already waiting for us. All we have to do is the following. 
#The run_options are not necessary but will produce a timeout in case something went wrong beforehand.
run_options=tf.RunOptions(timeout_in_ms=4000)
curr_data_batch,curr_target_batch=sess.run([data_batch,target_batch],options=run_options)
print(curr_data_batch)

[[  0.   0.   0.   0.]
 [  1.   1.   1.   1.]
 [  2.   2.   2.   2.]
 [  3.   3.   3.   3.]
 [  4.   4.   4.   4.]
 [  5.   5.   5.   5.]
 [  6.   6.   6.   6.]
 [  7.   7.   7.   7.]
 [  8.   8.   8.   8.]
 [  9.   9.   9.   9.]
 [ 10.  10.  10.  10.]
 [ 11.  11.  11.  11.]
 [ 12.  12.  12.  12.]
 [ 13.  13.  13.  13.]
 [ 14.  14.  14.  14.]]
added to the queue
starting to write into queue
try to enqueue  100  to  120


In [35]:
#Once we are done with our input pipeline we should stop all running threads before closing the session.
sess.run(queue.close(cancel_pending_enqueues=True))
coord.request_stop()
coord.join(threads)
sess.close()
#When you run this script and close the queue in the end it is possible that you get Enqueue operation was cancelled errors.
#This has to do with closing the queues while some threads still seem to be operating.
#However, if you do it like this no threads will survive and use up your GPU memory.
#If you run into weird CUDA memory errors,
#have a look at the running processes and make sure there is no zombie thread from your last run.

Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\OM\Anaconda2\envs\py35\lib\site-packages\tensorflow\python\client\session.py", line 1139, in _do_call
    return fn(*args)
  File "C:\Users\OM\Anaconda2\envs\py35\lib\site-packages\tensorflow\python\client\session.py", line 1121, in _run_fn
    status, run_metadata)
  File "C:\Users\OM\Anaconda2\envs\py35\lib\contextlib.py", line 66, in __exit__
    next(self.gen)
  File "C:\Users\OM\Anaconda2\envs\py35\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 466, in raise_exception_on_not_ok_status
    pywrap_tensorflow.TF_GetCode(status))
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[Node: fifo_queue_1_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](fifo_queue_1, _arg_Placeholder_4_0_0, _arg_Placeholder_5_0_1)]]

During handling of the above exception, another excep

In [13]:
def normalize(x):
    """
    Normalize a list of sample image data in the range of 0 to 1
    : x: List of image data.  The image shape is (32, 32, 3)
    : return: Numpy array of normalize data
    """
    # TODO: Implement Function
    from numpy import linalg as LA
    
    x=x/255
    return x
for im in images:
    im=normalize(im)

In [ ]:
#Since we have imbalanced data, we use SMOTE algorithm to over sample the training data
from imblearn.over_sampling import SMOTE

sm=SMOTE(random_state=12,ratio=0.1)
X_train_res,y_train_res=sm.fit_sample(X_train,y_train)
